In [1]:
import pandas as pd
import numpy as np

from subprocess import call

In [28]:
from wjhfinemap import *

In [3]:
sumstat = pd.read_csv('../new_gwas/CAD_29212778/CA448.txt.gz',sep='\t')

In [5]:
def merge_loci(sig_blocks):
    merge_blocks = sig_blocks.copy()
    while True:
        for i in merge_blocks.index:
            chrom, start, end = merge_blocks.loc[i, ["chr", "start", "end"]]
            sub_df = merge_blocks[
                (merge_blocks["chr"] == chrom)
                & (merge_blocks["end"] >= start)
                & (merge_blocks["start"] <= end)
            ]
            lead_idx = sub_df["lead_p"].idxmin()
            merge_blocks.loc[i] = (
                sub_df.loc[lead_idx, "lead_snp"],
                sub_df.loc[lead_idx, "lead_p"],
                chrom,
                sub_df["start"].min(),
                sub_df["end"].max(),
            )
        merge_blocks = merge_blocks.drop_duplicates()
        overlap = 0
        for i in merge_blocks.index:
            chrom, start, end = merge_blocks.loc[i, ["chr", "start", "end"]]
            sub_df = merge_blocks[
                (merge_blocks["chr"] == chrom)
                & (merge_blocks["end"] >= start)
                & (merge_blocks["start"] <= end)
            ]
            if len(sub_df) > 1:
                overlap = 1
                break
        if not overlap:
            break

    merge_blocks = merge_blocks.drop_duplicates()
    return merge_blocks

In [6]:
def get_distance_loci(sumstat, sig_cutoff=5e-8, chunk_size=500000):
    sig_df = sumstat[sumstat["P"] <= sig_cutoff].copy().sort_values("P")
    sig_df["rsID"] = sig_df["CHR"].astype(str) + "-" + sig_df["BP"].astype(str)
    sig_blocks = pd.DataFrame(columns=["lead_snp", "lead_p", "chr", "start", "end"])
    ith = 0
    while len(sig_df) > 0:
        chrom, lead_bp, lead_snp, lead_p = sig_df.iloc[0][
            ["CHR", "BP", "rsID", "P"]
        ].values
        sig_blocks.loc[ith] = (
            lead_snp,
            lead_p,
            chrom,
            lead_bp - chunk_size,
            lead_bp + chunk_size,
        )
        sig_df = sig_df[
            ~(
                (sig_df["CHR"] == chrom)
                & (sig_df["BP"] >= lead_bp - chunk_size)
                & (sig_df["BP"] <= lead_bp + chunk_size)
            )
        ]
        ith += 1
    sig_blocks = sig_blocks.sort_values(["chr", "start", "end"])
    sig_blocks.index = range(len(sig_blocks))
    return sig_blocks

In [14]:
loci = get_distance_loci(sumstat)

In [15]:
loci = merge_loci(loci)

In [16]:
loci

,lead_snp,lead_p,chr,start,end
0,1-2252205,1.326000e-08,1,1752205,2752205
1,1-3325912,9.974000e-10,1,2825912,3825912
2,1-38456106,4.129000e-10,1,37956106,38956106
3,1-55505647,1.855000e-22,1,55005647,56005647
4,1-56966350,8.038000e-28,1,56466350,57466350
...,...,...,...,...,...
168,20-57714025,7.960000e-10,20,57214025,58214025
169,21-30533076,4.164000e-09,21,30033076,31033076
170,21-35593827,2.579000e-33,21,35093827,36093827
171,22-24672718,3.730000e-09,22,24172718,25172718


In [18]:
sub_df = sumstat[(sumstat['CHR']==1)&(sumstat['BP']>=1752205)&(sumstat['BP']<=2752205)]

In [19]:
sub_df

,CHR,BP,rsID,MAF,EA,NEA,BETA,SE,P,Zscore
1637,1,1752955,rs4648726,0.1016,T,C,0.0073,0.0105,0.48380,0.695238
1638,1,1753686,rs12141314,0.1730,A,G,-0.0178,0.0074,0.01624,-2.405405
1639,1,1757221,rs2144688,0.0654,A,G,-0.0136,0.0122,0.26400,-1.114754
1640,1,1759026,rs9786963,0.0968,T,C,-0.0104,0.0109,0.33630,-0.954128
1641,1,1759054,rs10907187,0.2564,A,G,0.0118,0.0065,0.06786,1.815385
...,...,...,...,...,...,...,...,...,...,...
3878,1,2751107,rs141899166,0.0104,A,G,-0.0038,0.0298,0.89940,-0.127517
3879,1,2751237,rs72646052,0.1158,A,G,0.0031,0.0094,0.74340,0.329787
3880,1,2751364,rs75404797,0.1162,T,C,0.0026,0.0094,0.78240,0.276596
3881,1,2751880,rs116166994,0.0063,A,G,-0.0059,0.0422,0.88850,-0.139810


In [26]:
def ref_intersec(bim_prefix, out_prefix, sumstat, work_dir, annotate_eaf=True):
    out_prefix = f'{work_dir}/{out_prefix}'
    call(f'mkdir -p {work_dir}', shell=True)
    chrom, start, end = sumstat['CHR'].values[0], sumstat['BP'].min(), sumstat['BP'].max()
    with open(f'{out_prefix}.region', 'w') as f:
        f.write(f'{chrom}\t{start}\t{end}\tregion')

    call(f'''plink --bfile {bim_prefix} \
             --extract range {out_prefix}.region \
             --make-bed --out {out_prefix}.orig \
             > {out_prefix}.orig.log''', shell=True)

    bim = pd.read_csv(f"{out_prefix}.orig.bim", delim_whitespace=True, header=None)
    bim[1] = bim[3].astype(str) + "-" + bim[5] + "-" + bim[4]
    bim.to_csv(f"{out_prefix}.orig.bim", sep="\t", index=False, header=False)
    refalt_snps = bim[1].values
    sub_df = sumstat.copy()
    sub_df["refalt"] = (
        sub_df["BP"].astype(str) + "-" + sub_df["NEA"] + "-" + sub_df["EA"]
    )
    sub_df["altref"] = (
        sub_df["BP"].astype(str) + "-" + sub_df["EA"] + "-" + sub_df["NEA"]
    )

    sub_df = sub_df[
        (sub_df["refalt"].isin(refalt_snps)) | (sub_df["altref"].isin(refalt_snps))
    ]
    sub_df = sub_df.reset_index(drop=True)
    b = sub_df[["EA", "NEA"]].copy()
    sub_df["EA"] = b["NEA"].where(sub_df["altref"].isin(refalt_snps), b["EA"])
    sub_df["NEA"] = b["EA"].where(sub_df["altref"].isin(refalt_snps), b["NEA"])
    sub_df["BETA"] = -sub_df["BETA"].where(
        sub_df["altref"].isin(refalt_snps), sub_df["BETA"]
    )
    sub_df["Zscore"] = -sub_df["Zscore"].where(
        sub_df["altref"].isin(refalt_snps), sub_df["Zscore"]
    )

    sub_df["snp"] = sub_df["BP"].astype(str) + "-" + sub_df["NEA"] + "-" + sub_df["EA"]
    sub_df["snp"].to_csv(f"{out_prefix}.snplist", sep="\t", header=False, index=False)

    call(
        f'''plink --bfile {out_prefix}.orig \
            --extract {out_prefix}.snplist \
            --make-bed \
            --keep-allele-order \
            --out {out_prefix} > {out_prefix}.log''',
        shell=True,
    )

    if annotate_eaf:
        call(
            f"plink --bfile {out_prefix} --freq --out {out_prefix} > {out_prefix}.frq.log",
            shell=True,
        )
        freq = pd.read_csv(f"{out_prefix}.frq", delim_whitespace=True)
        sub_df["EAF"] = freq["MAF"].where(sub_df["EA"] == freq["A1"], 1 - freq["MAF"])
    sub_df = sub_df.drop(["refalt", "altref"], axis=1)
    sub_df.to_csv(f"{out_prefix}.sumstat", sep="\t", index=False)
    call(f'rm {out_prefix}.orig.bim', shell=True)



In [27]:
ref_intersec('/h/jianhua/REF/1000G/20130502/pop_plink/EUR.chr1', '1-2252205', sub_df, '1-2252205', )

In [4]:
for chrom, sub_df in sumstat.groupby('CHR'):
    if len(sub_df[sub_df['P']<=5e-8]) > 0:
        print(chrom)
        break

1


In [10]:
ref_intersec('./UK10K.chr1', 'UK10K.chr1.overlap', sub_df)

In [59]:
sub_df

,CHR,BP,rsID,MAF,EA,NEA,BETA,SE,P,Zscore
0,1,752566,rs3094315,0.1918,A,G,-0.0030,0.0082,0.7161,-0.365854
1,1,752721,rs3131972,0.2007,A,G,0.0023,0.0082,0.7809,0.280488
2,1,753405,rs3115860,0.1655,A,C,0.0047,0.0088,0.5957,0.534091
3,1,753541,rs2073813,0.1582,A,G,-0.0049,0.0088,0.5794,-0.556818
4,1,754182,rs3131969,0.1729,A,G,-0.0049,0.0087,0.5786,-0.563218
...,...,...,...,...,...,...,...,...,...,...
612494,1,249216538,rs11205398,0.0304,A,G,-0.0206,0.0174,0.2359,-1.183908
612495,1,249217820,rs35622786,0.0424,T,C,0.0105,0.0153,0.4940,0.686275
612496,1,249218281,rs12066487,0.0312,A,G,-0.0188,0.0173,0.2795,-1.086705
612497,1,249220525,rs4356123,0.0601,A,G,0.0188,0.0144,0.1893,1.305556


In [18]:
a = pd.read_csv('./CA448.chr1.500.UK10K.ma',sep=' ')

In [24]:
a[a['p']<=5e-8]

,SNP,A1,A2,freq,b,se,p,N
2494,2247399-T-C,C,T,0.7979,-0.0380,0.0069,4.063000e-08,10000
2496,2247985-T-C,C,T,0.7981,-0.0379,0.0069,4.370000e-08,10000
2508,2251982-C-T,T,C,0.1933,-0.0403,0.0071,1.555000e-08,10000
2509,2252191-A-G,G,A,0.1941,0.0395,0.0071,2.978000e-08,10000
2510,2252205-C-T,T,C,0.1437,-0.0469,0.0082,1.326000e-08,10000
...,...,...,...,...,...,...,...,...
510632,222949258-G-T,T,G,0.7577,-0.0473,0.0069,7.821000e-12,10000
510633,222949354-T-C,C,T,0.7545,0.0467,0.0069,1.093000e-11,10000
510634,222949898-A-G,G,A,0.7618,0.0467,0.0069,1.646000e-11,10000
510635,222950119-G-A,A,G,0.7605,-0.0468,0.0069,1.507000e-11,10000


In [21]:
a[a['p']<=5e-8]['SNP'].to_csv('./sig.snps',sep='\t',index=False, header=False)

In [22]:
def cojo_slct(bfile, sumstat, sample_size, out_prefix):
    sumstat = pd.read_csv(sumstat, sep='\t')
    sumstat['N'] = sample_size
    sumstat = sumstat[['snp','EA','NEA','EAF','BETA','SE','P','N']]
    sumstat.columns = ['SNP','A1','A2','freq','b','se','p','N']
    sumstat.to_csv(f'{out_prefix}.ma', sep=' ', index=False)
    cmd = f'''gcta64 --bfile {bfile} \
              --extract ./sig.snps \
              --cojo-p 5e-8 \
              --cojo-wind 500 \
              --cojo-collinear 0.9 \
              --maf 0.01 \
              --cojo-file {out_prefix}.ma \
              --cojo-slct \
              --out {out_prefix}'''
    call(cmd, shell=True)

In [23]:
cojo_slct('./UK10K.chr1.overlap', './UK10K.chr1.overlap.sumstat', 10000, './CA448.chr1.500.UK10K.extractsnps')

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version 1.93.2 beta Linux
* (C) 2010-present, Jian Yang, The University of Queensland
* Please report bugs to Jian Yang <jian.yang.qt@gmail.com>
*******************************************************************
Analysis started at 17:31:33 CST on Wed Nov 09 2022.
Hostname: CNV

Accepted options:
--bfile ./UK10K.chr1.overlap
--extract ./sig.snps
--cojo-p 5e-08
--cojo-wind 500
--cojo-collinear 0.9
--maf 0.01
--cojo-file ./CA448.chr1.500.UK10K.extractsnps.ma
--cojo-slct
--out ./CA448.chr1.500.UK10K.extractsnps


Reading PLINK FAM file from [./UK10K.chr1.overlap.fam].
1927 individuals to be included from [./UK10K.chr1.overlap.fam].
Reading PLINK BIM file from [./UK10K.chr1.overlap.bim].
588738 SNPs to be included from [./UK10K.chr1.overlap.bim].
Reading a list of SNPs from [./sig.snps].
619 SNPs are extracted from [./sig.snps].
Reading PLINK BED file from [./UK10K.chr1.overla